In [1]:
import os
import re
from copy import deepcopy
from pathlib import Path

import numpy as np
import scipy.constants

import cheetah

In [2]:
os.environ["LCLS_LATTICE"] = str((Path(".").absolute().parent / "lcls-lattice"))
os.environ["LCLS_LATTICE"]

'/Users/jankaiser/Documents/DESY/lcls-lattice'

In [3]:
lattice_file_path = (
    Path("$LCLS_LATTICE") / "bmad" / "models" / "cu_hxr" / "cu_hxr.lat.bmad"
)
lattice_file_path

PosixPath('$LCLS_LATTICE/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [4]:
resolved_lattice_file_path = Path(
    *[
        os.environ[part[1:]] if part.startswith("$") else part
        for part in lattice_file_path.parts
    ]
)
resolved_lattice_file_path

PosixPath('/Users/jankaiser/Documents/DESY/lcls-lattice/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [5]:
lines = cheetah.bmad.read_clean_lines(resolved_lattice_file_path)
lines[:20]

['beginning[beta_a] =  5.91253676811640894e+000',
 'beginning[alpha_a] =  3.55631307633660354e+000',
 'beginning[beta_b] =  5.91253676811640982e+000',
 'beginning[alpha_b] =  3.55631307633660398e+000',
 'beginning[e_tot] = 6e6',
 'parameter[geometry] = open',
 'parameter[particle] = electron',
 'beginning[theta_position] = -35*pi/180',
 'beginning[z_position] = 3050.512000 - 1032.60052',
 'beginning[x_position] = 10.44893',
 'setsp = 0',
 'setcus = 0',
 'setal = 0',
 'setda = 0',
 'setxleap2 = 0',
 'sethxrss = 0',
 'setsxrss = 0',
 'setcbxfel = 0',
 'setpepx = 0',
 'intgsx = 30.0']

In [6]:
merged_lines = cheetah.bmad.merge_delimiter_continued_lines(
    lines, delimiter="&", remove_delimiter=True
)
merged_lines = cheetah.bmad.merge_delimiter_continued_lines(
    merged_lines, delimiter=",", remove_delimiter=False
)
merged_lines = cheetah.bmad.merge_delimiter_continued_lines(
    merged_lines, delimiter="{", remove_delimiter=False
)
len(lines), len(merged_lines)

(14409, 12215)

In [7]:
property_assignment_pattern = r"[a-z0-9_]+\[[a-z0-9_%]+\]\s*=.*"
variable_assignment_pattern = r"[a-z0-9_]+\s*=.*"
element_definition_pattern = r"[a-z0-9_]+\s*\:.*"
line_definition_pattern = r"[a-z0-9_]+\s*\:\s*line\s*=\s*\(.*\)"
overlay_definition_pattern = r"[a-z0-9_]+\s*\:\s*overlay\s*=\s*\{.*"
use_line_pattern = r"use\s*\,\s*[a-z0-9_]+"

num_successful = 0
num_property_assignment = 0
num_variable_assignment = 0
num_element_definition = 0
num_line_definition = 0
num_overlay_definition = 0
num_use_line = 0
for line in merged_lines:
    if re.fullmatch(property_assignment_pattern, line):
        num_successful += 1
        num_property_assignment += 1
    elif re.fullmatch(variable_assignment_pattern, line):
        num_successful += 1
        num_variable_assignment += 1
    elif re.fullmatch(line_definition_pattern, line):
        num_successful += 1
        num_line_definition += 1
    elif re.fullmatch(overlay_definition_pattern, line):
        num_successful += 1
        num_overlay_definition += 1
    elif re.fullmatch(element_definition_pattern, line):
        num_successful += 1
        num_element_definition += 1
    elif re.fullmatch(use_line_pattern, line):
        num_successful += 1
        num_use_line += 1
    else:
        print(line)
        break

print("")
print("######################################")
print(f"num_successful: {num_successful} / {len(merged_lines)}")
print("--------------------------------------")
print(f"{num_property_assignment = }")
print(f"{num_variable_assignment = }")
print(f"{num_element_definition = }")
print(f"{num_line_definition = }")
print(f"{num_overlay_definition = }")
print(f"{num_use_line = }")
print("######################################")


######################################
num_successful: 12215 / 12215
--------------------------------------
num_property_assignment = 4256
num_variable_assignment = 1684
num_element_definition = 4859
num_line_definition = 1309
num_overlay_definition = 106
num_use_line = 1
######################################


In [8]:
def evaluate_expression(expression: str, context: dict) -> dict:
    """Evaluate an expression in the context of a dictionary of variables."""

    # Try reading the expression as an integer
    try:
        return int(expression)
    except ValueError:
        pass

    # Try reading the expression as a float
    try:
        return float(expression)
    except ValueError:
        pass

    # Check against allowed keywords
    if expression in ["open", "electron", "t", "f"]:
        return expression

    # Check against previously defined variables
    if expression in context:
        return context[expression]

    # Evaluate as a mathematical expression
    try:
        # Translate getting an element's length to Cheetah's syntax
        expression = expression.replace("[l]", ".length")
        # Surround expressions in bracks with quotes
        expression = re.sub(r"\[([a-z0-9_%]+)\]", r"['\1']", expression)

        return eval(expression, context)
    except Exception as e:
        print(expression)
        print(context["dsp12hc"])
        raise e

    print(expression)
    result = "foobar"

    return result

In [9]:
def assign_property(line: str, context: dict) -> dict:
    """Assign a property to the context."""
    pattern = r"([a-z0-9_]+)\[([a-z0-9_%]+)\]\s*=(.*)"
    match = re.fullmatch(pattern, line)

    object_name = match.group(1).strip()
    property_name = match.group(2).strip()
    property_expression = match.group(3).strip()  # TODO: Evaluate expression first

    expression_result = evaluate_expression(property_expression, context)

    if object_name not in context:
        context[object_name] = {}
    context[object_name][property_name] = expression_result

    return context

In [10]:
def assign_variable(line: str, context: dict) -> dict:
    """Assign a variable to the context."""
    pattern = r"([a-z0-9_]+)\s*=(.*)"
    match = re.fullmatch(pattern, line)

    variable_name = match.group(1).strip()
    variable_expression = match.group(2).strip()  # TODO: Evaluate expression first

    context[variable_name] = evaluate_expression(variable_expression, context)

    return context

In [11]:
def validate_understood_properties(understood: list[str], properties: dict) -> None:
    """
    Validate that all properties are understood. This function primarily ensures that
    properties not understood by Cheetah are not ignored silently.
    """
    for property in properties:
        assert property in understood, (
            f"Property {property} with value {properties[property]} for element"
            f" type {properties['element_type']} is currently not understood."
        )

In [24]:
def convert_element(name: str, properties: dict):
    """
    Convert parsed element dictionary from a Bmad lattice file to a Cheetah element.
    """
    if properties["element_type"] == "drift" or properties["element_type"] == "pipe":
        validate_understood_properties(
            ["element_type", "l", "type", "descrip"], properties
        )
        return cheetah.Drift(name=name, length=properties["l"])
    elif (
        properties["element_type"] == "marker"
        or properties["element_type"] == "monitor"
        or properties["element_type"] == "instrument"
    ):
        validate_understood_properties(["element_type", "type"], properties)

        # TODO: Remove the length if by adding markers to Cheeath
        return cheetah.Drift(name=name, length=0.0)
    elif properties["element_type"] == "quadrupole":
        # TODO: Aperture for quadrupoles?
        validate_understood_properties(
            ["element_type", "l", "k1", "type", "aperture"], properties
        )
        return cheetah.Quadrupole(
            name=name, length=properties["l"], k1=properties["k1"]
        )
    else:
        print(
            f"WARNING: Element of type {properties['element_type']} cannot be converted"
            " correctly. Using drift section instead."
        )
        # TODO: Remove the length if by adding markers to Cheeath
        return cheetah.Drift(
            name=name, length=properties["l"] if "l" in properties else 0.0
        )

In [25]:
def define_element(line: str, context: dict) -> dict:
    """Define an element in the context."""
    pattern = r"([a-z0-9_]+)\s*\:\s*([a-z0-9_]+)(\,(.*))?"
    match = re.fullmatch(pattern, line)

    element_name = match.group(1).strip()
    element_type = match.group(2).strip()

    if element_type in context:
        element_properties = deepcopy(context[element_type])
    element_properties = {"element_type": element_type}

    if match.group(3) is not None:
        element_properties_string = match.group(4).strip()

        property_pattern = r"([a-z0-9_]+\s*\=\s*\"[^\"]+\"|[a-z0-9]+\s*\=\s*[^\=\,\"]+)"
        property_matches = re.findall(property_pattern, element_properties_string)

        for property_string in property_matches:
            property_string = property_string.strip()

            property_name, property_expression = property_string.split("=")
            property_name = property_name.strip()
            property_expression = property_expression.strip()

            element_properties[property_name] = evaluate_expression(
                property_expression, context
            )

    context[element_name] = convert_element(element_name, element_properties)

    return context

In [26]:
def define_line(line: str, context: dict) -> dict:
    """Define a beam line in the context."""
    pattern = r"([a-z0-9_]+)\s*\:\s*line\s*=\s*\((.*)\)"
    match = re.fullmatch(pattern, line)

    line_name = match.group(1).strip()
    line_elements_string = match.group(2).strip()

    line_elements = []
    for element_name in line_elements_string.split(","):
        element_name = element_name.strip()

        line_elements.append(element_name)

    context[line_name] = line_elements

    return context

In [27]:
def define_overlay(line: str, context: dict) -> dict:
    """Define an overlay in the context."""
    pattern = r"([a-z0-9_]+)\s*\:\s*overlay\s*=\s*\{(.*)"
    match = re.fullmatch(pattern, line)

    overlay_name = match.group(1).strip()
    overlay_definition_string = match.group(2).strip()

    context[overlay_name] = overlay_definition_string

    return context

In [28]:
def parse_use_line(line: str, context: dict) -> dict:
    """Parse a use line."""
    pattern = r"use\s*\,\s*([a-z0-9_]+)"
    match = re.fullmatch(pattern, line)

    use_line_name = match.group(1).strip()
    context["__use__"] = use_line_name

    return context

In [29]:
context = {
    "pi": scipy.constants.pi,
    "c_light": scipy.constants.c,
    "emass": scipy.constants.electron_mass,
    "sqrt": np.sqrt,
    "asin": np.arcsin,
    "sin": np.sin,
    "cos": np.cos,
    "raddeg": scipy.constants.degree,
}
for line in merged_lines:
    if re.fullmatch(property_assignment_pattern, line):
        context = assign_property(line, context)
    elif re.fullmatch(variable_assignment_pattern, line):
        context = assign_variable(line, context)
    elif re.fullmatch(line_definition_pattern, line):
        context = define_line(line, context)
    elif re.fullmatch(overlay_definition_pattern, line):
        context = define_overlay(line, context)
    elif re.fullmatch(element_definition_pattern, line):
        context = define_element(line, context)
    elif re.fullmatch(use_line_pattern, line):
        context = parse_use_line(line, context)

TypeError: 'Quadrupole' object does not support item assignment

In [ ]:
context

{'pi': 3.141592653589793,
 'c_light': 299792458.0,
 'emass': 9.1093837015e-31,
 'sqrt': <ufunc 'sqrt'>,
 'asin': <ufunc 'arcsin'>,
 'sin': <ufunc 'sin'>,
 'cos': <ufunc 'cos'>,
 'raddeg': 0.017453292519943295,
 'beginning': {'beta_a': 5.912536768116409,
  'alpha_a': 3.5563130763366035,
  'beta_b': 5.91253676811641,
  'alpha_b': 3.556313076336604,
  'e_tot': 6000000.0,
  'theta_position': -0.6108652381980153,
  'z_position': 2017.9114800000002,
  'x_position': 10.44893},
 'parameter': {'geometry': 'open', 'particle': 'electron'},
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'),
  '__build_class__': <function __build_class__>,
  '__import__': <function __import__>,
  'a

In [ ]:
line = 'ssp1h: pipe, l = 0.05, type ="@1,1.38s3.00", descrip = "deferred sextupole"'
line

'ssp1h: pipe, l = 0.05, type ="@1,1.38s3.00", descrip = "deferred sextupole"'

In [ ]:
pattern = r"([a-z0-9_]+)\s*"
match = re.fullmatch(pattern, line)
match[3]

TypeError: 'NoneType' object is not subscriptable